# Conversion of raw files .ddf files to csv and nc

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
from utils import read_and_combine_dts_files
from utils import convert_ddf_to_monthly_csv

#### Converting the raw ddf files to monthly csv files

In [5]:
in_ddf_directory = r'/Users/jpnousu/DTS_data/RAW_DATA_DDF/'
out_csv_directory = r'/Users/jpnousu/DTS_data/RAW_DATA_CSV/'
output_nc_file = '/Users/jpnousu/DTS_data/PROCESSED_DATA/pallas_dts_data.nc'
dates_from_filenames = True

convert_ddf_to_monthly_csv(in_ddf_directory, out_csv_directory, dates_from_filenames=dates_from_filenames)

Saved: /Users/jpnousu/DTS_data/RAW_DATA_CSV_DUMMY/channel 1 dts jan 2023.csv


#### Reading and combining the csv files into a single pd DataFrame

In [6]:
df_pivot = read_and_combine_dts_files(directory=out_csv_directory, dates_from_filenames=dates_from_filenames)
df_pivot.head()

length (m),60.603,62.633,64.662,66.691,68.720,70.749,72.778,74.808,76.837,78.866,...,1921.370,1923.399,1925.429,1927.458,1929.487,1931.516,1933.545,1935.574,1937.604,1939.633
2023-01-17 13:30:35,1.288,1.216,1.456,1.231,1.180,1.427,1.289,1.264,1.422,1.119,...,4.220,4.333,4.245,4.209,4.317,4.243,4.214,4.236,4.224,4.315
2023-01-17 14:00:34,1.358,1.279,1.295,1.062,1.295,1.459,1.166,1.477,1.342,1.448,...,4.386,4.332,4.257,4.314,4.433,4.369,4.496,4.367,4.482,4.343


#### Creating xarray dataset and netCDF file (.nc) from the pd DataFrame

In [7]:
# Convert the pivoted DataFrame to xarray Dataset
xr_dataset = xr.Dataset(
    data_vars={
        'T': (['time', 'x'], df_pivot.values)
    },
    coords={
        'time': df_pivot.index.values,
        'x': df_pivot.columns.values
    }
)
# Adding descriptions and units
xr_dataset['T'].attrs['units'] = '°C'
xr_dataset['T'].attrs['description'] = 'Temperature in degrees Celsius'
xr_dataset['x'].attrs['units'] = 'm'
xr_dataset['x'].attrs['description'] = 'Length in meters'

# Save the xarray Dataset as a NetCDF file
xr_dataset.to_netcdf(output_nc_file)